In [7]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os

In [9]:
def read_data(input_file):
    rf = open(input_file, 'r')
    lines = []; words = []; labels = []
    for line in rf:
        word = line.strip().split(' ')[0]
        label = line.strip().split(' ')[-1]
        if len(line.strip()) == 0 and words[-1] == '.':
            l = ' '.join([label for label in labels if len(label) > 0])
            w = ' '.join([word for word in words if len(word) > 0])
            lines.append((l, w))
            words = []
            labels = []
        words.append(word)
        labels.append(label)
    labels_list=[]
    for l in lines:
        labels_list.append(l[0])
    return labels_list, lines
   
og_labels, og_lines =  read_data("./baselines/bert_baseline/original_conll03/original_test_predictions.txt")
prop_labels, prop_lines =  read_data("./baselines/bert_baseline/proposed_conll03/proposed_test_predictions.txt")

In [10]:
"""
finding incongruent sequences like B-PER I-LOC
currently only works for entities with two words
"""
def incongruent_label(labels):
    incong=[]
    for l in labels:
        ll=l.split(' ')
        for w in range(len(ll)):
            if ll[w]=='O':
                pass
            else:
                if ll[w].split('-')[0]=='B':
                    if ll[w+1].split('-')[0]=='I':
                        if  ll[w].split('-')[1]!=ll[w+1].split('-')[1]:
                            incong.append((ll[w],ll[w+1]))
    return incong

og_incongruent=incongruent_label(og_labels)
prop_incongruent=incongruent_label(prop_labels)
print(len(og_incongruent),og_incongruent)

print(len(prop_incongruent),prop_incongruent)

52 [('B-ORG', 'I-PER'), ('B-ORG', 'I-PER'), ('B-PER', 'I-ORG'), ('B-LOC', 'I-MISC'), ('B-ORG', 'I-MISC'), ('B-ORG', 'I-MISC'), ('B-LOC', 'I-MISC'), ('B-LOC', 'I-MISC'), ('B-ORG', 'I-MISC'), ('B-LOC', 'I-ORG'), ('B-ORG', 'I-MISC'), ('B-PER', 'I-ORG'), ('B-ORG', 'I-PER'), ('B-LOC', 'I-MISC'), ('B-MISC', 'I-ORG'), ('B-ORG', 'I-MISC'), ('B-ORG', 'I-LOC'), ('B-LOC', 'I-ORG'), ('B-MISC', 'I-LOC'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-MISC'), ('B-ORG', 'I-LOC'), ('B-PER', 'I-LOC'), ('B-PER', 'I-LOC'), ('B-MISC', 'I-ORG'), ('B-LOC', 'I-PER'), ('B-LOC', 'I-ORG'), ('B-MISC', 'I-LOC'), ('B-MISC', 'I-ORG'), ('B-LOC', 'I-MISC'), ('B-MISC', 'I-ORG'), ('B-MISC', 'I-ORG'), ('B-MISC', 'I-ORG'), ('B-PER', 'I-ORG'), ('B-PER', 'I-LOC'), ('B-MISC', 'I-ORG'), ('B-LOC', 'I-MISC'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-MISC', 'I-LOC'), ('B-MISC', 'I-LOC'), ('B-ORG', 'I-MISC'), ('B-LOC', 'I-MISC'), ('B-ORG', 'I-MISC'), ('B-LOC', 